In [1]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [1]:
import galois
import numpy as np
import time

In [2]:
#definisco un campo di Galois come nell'esempi
field = galois.GF(2**3)

In [3]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 3 #le righe
n_k = 4 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [4]:
#questa è con l'esempio di 4.3
A = field([[6, 6, 3, 0],
           [7, 5, 0, 7],
           [5, 0, 6, 1]])

In [6]:
#ho verificato che funziona pure con i random
"""A = field(np.random.randint(0, field.order, size=(k, n_k)))
print(A)"""

'A = field(np.random.randint(0, field.order, size=(k, n_k)))\nprint(A)'

In [5]:
matrice_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrice_bin1 = field(matrice_bin1)

matrice_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrice_bin2 = field(matrice_bin2)

In [6]:
A_primo = field(matrice_bin1 @ A @ matrice_bin2)
print(A_primo)

[[5 1 0 6]
 [6 0 6 3]
 [7 7 5 0]]


In [9]:
"""A_primo = field([[2,2,0,3],
           [2,0,2,3],
           [0,3,1,2]])"""

'A_primo = field([[2,2,0,3],\n           [2,0,2,3],\n           [0,3,1,2]])'

In [7]:
#definito funzione ordinamento lessicografico riga (ordino ogni riga e definisco ordine degli indici) e gestiti casi di 
#conflitto. L'argomento colonne è quello che ci indica in che ordine verificare i conflitti 
#ex: colonne=[0,1,2] --> ordino per colonna 0, se c'è conflitto tra due vedo la colonna 1 e cosi via

#faccio ordinamento crescente dei valori all'interno della riga e torno matrice con righe ordinate (ordinato).
#da questa matrice faccio l'ordinamento elemento a elemento posizionale corrispondente e definisco quindi gli indici 
#riordinati. ex: [1 0 2] --> prima la riga ORIGINALE 1 poi riga ORIGINALE 0 e così via. Per originale intendo "matrice"

#questa è la funzione che gli fornisco la matrice ordinata all'interno delle righe e lei pme la ordina per righe
def ordina_matrice(matrice):
    print(matrice)
    #righe matrice
    n = matrice.shape[0]  
    #contatore confronti
    confronti = 0  
    #array indici originali
    indici = list(np.arange(n))  

    #bubble sort per conteggio confronti
    for i in range(n):
        for j in range(i + 1, n):
            print(f"Confronto tra riga {i} e riga {j}")
            #confronto per primo el riga poi secondo e cosi via
            for k in range(matrice.shape[1]):  
                confronti += 1
                #sono ordinate tra loro, vado avanti
                if matrice[indici[i], k] < matrice[indici[j], k]: 
                    print(f"Riga {i} ({matrice[indici[i]]}) è minore di Riga {j} ({matrice[indici[j]]})")
                    break
                #devo mettere la riga sopra quella con cui è confrontata e andare avanti
                elif matrice[indici[i], k] > matrice[indici[j], k]:
                    print(f"Riga {i} ({matrice[indici[i]]}) è maggiore di Riga {j} ({matrice[indici[j]]})")
                    #sposto indice sopra
                    indici[i], indici[j] = indici[j], indici[i]
                    print("Indici dopo lo spostamento:", indici)
                    break 
                if k == (n_k - 1):
                    print("Algoritmo ha almeno due righe uguali --> fallisco")
                    return [], confronti
    print("Indici delle righe ordinate:", indici)
    print("Numero totale di confronti:", confronti)
    
    return indici, confronti

def ordina_matrice_ottimizzata(matrice):
    print("Entro in ordina matrice ottimizzata")
    
    n = matrice.shape[0]
    n_colonne = matrice.shape[1]
    
    # Array indici originali
    indici = np.arange(n)
    indici1 = np.arange(n)

    # Timing
    start = time.perf_counter()

    # Bubble sort sostituito da numpy.argsort per efficienza
    indici = np.lexsort([matrice[:, i] for i in range(n_colonne - 1, -1, -1)])  # Ordinamento completo
    end = time.perf_counter()
    tempo = (end - start) * 1000

    return indici, 0
   



def ordina_per_riga(matrice):
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
        
    ordinato = np.array(ordinato)
    
    indici_ordinati, num_confronti = ordina_matrice(ordinato)
    if len(indici_ordinati) == 0:
        return []
    print("Matrice ordinata per righe: ",matrice[indici_ordinati])
    return matrice[indici_ordinati]

#ordino poi per il primo elemento della colonna e in caso di conflitti li ho gestiti. 
def ordina_per_colonna(matrice, righe):
    if len(matrice) == 0:
        return []
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
    return matrice[:, indici_ordinamento]

#qui faccio il test
def test_algoritmo(matrice1, matrice2):
    righe = range(k)
    print("Matrice1: ", matrice1)

    ordinata1 = ordina_per_colonna(ordina_per_riga(matrice1), righe)
    print("\nOrdinata 1: ",ordinata1)

    print("Matrice2: ", matrice2)
    ordinata2 = ordina_per_colonna(ordina_per_riga(matrice2), righe)
    print("\nOrdinata2: ",ordinata2)     

    if np.array_equal(ordinata1, ordinata2):
        print("Le due matrici hanno la stessa CF.")
        return True
    else:
        print("Le due matrici hanno CF diversa")
        return False   



In [8]:
test_algoritmo(A, A_primo)

Matrice1:  [[6 6 3 0]
 [7 5 0 7]
 [5 0 6 1]]
[[0 3 6 6]
 [0 5 7 7]
 [0 1 5 6]]
Confronto tra riga 0 e riga 1
Riga 0 ([0 3 6 6]) è minore di Riga 1 ([0 5 7 7])
Confronto tra riga 0 e riga 2
Riga 0 ([0 3 6 6]) è maggiore di Riga 2 ([0 1 5 6])
Indici dopo lo spostamento: [np.int64(2), np.int64(1), np.int64(0)]
Confronto tra riga 1 e riga 2
Riga 1 ([0 5 7 7]) è maggiore di Riga 2 ([0 3 6 6])
Indici dopo lo spostamento: [np.int64(2), np.int64(0), np.int64(1)]
Indici delle righe ordinate: [np.int64(2), np.int64(0), np.int64(1)]
Numero totale di confronti: 6
Matrice ordinata per righe:  [[5 0 6 1]
 [6 6 3 0]
 [7 5 0 7]]

Ordinata 1:  [[0 1 5 6]
 [6 0 6 3]
 [5 7 7 0]]
Matrice2:  [[5 1 0 6]
 [6 0 6 3]
 [7 7 5 0]]
[[0 1 5 6]
 [0 3 6 6]
 [0 5 7 7]]
Confronto tra riga 0 e riga 1
Riga 0 ([0 1 5 6]) è minore di Riga 1 ([0 3 6 6])
Confronto tra riga 0 e riga 2
Riga 0 ([0 1 5 6]) è minore di Riga 2 ([0 5 7 7])
Confronto tra riga 1 e riga 2
Riga 1 ([0 3 6 6]) è minore di Riga 2 ([0 5 7 7])
Indici delle

True